## Langchain에서 사용가능한 메모리 5가지
(당연히 다른 메모리들 더 있음.)

1. ConversationBufferMemory
- 대화 내용 전체를 저장 -> 길어지면 계속 쌓이기에 비효율적
- text completion할 때 유용
- 예측 혹은 텍스트 자동 완성
- chat 모델로 활용 시 ai, human 메시지 다 필요

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

memory.save_context({"input":"Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})

2. ConversationBufferWindowMemory
- 대화 특정 부분만 저장. 제일 최근 정보만 저장하기에 오래된건 크기에 따라 폐기 됨.   
  -> 메모리 특정 크기로 유지 가능.
- 전체가 아닌 최근 대화에만 집중을 함. -> 오래된건 기억 못함.

In [1]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4  # 메모리 저장 개수
)

def add_message(input, output):
    memory.save_context({"input":input}, {"output": output})


add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)

In [ ]:
memory.load_memory_variables({})

In [3]:
add_message(5,5)

#### 여기 부터는 메모리 실행 시 비용 발생.

3. ConversationSummaryMemory
- 모든 문장을 저장하는 것이 아닌 대화를 요약함.   
-> 매우 긴 문장일 경우 편리함.
- 처음에 많은 토큰을 요할 수 있음. 하지만 문장이 길경우 요약하기에 더 효율적임.

In [5]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input":input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm a Nicolas, I live in South Korea", "Wow that is so cool!")

In [6]:
add_message("South Korea is so pretty", "I wish I could go!!")

In [7]:
get_history()

{'history': 'Nicolas from South Korea introduces himself to the AI, who responds enthusiastically. The human mentions how pretty South Korea is, and the AI expresses a desire to visit.'}

4. ConversationSummaryBufferMemory
- summary + buffer : 요약과 최근 메모리 저장
- 메시지 수 저장함.
- limit에 다다르면, 잊는 대신 오래된 메시지를 요약함.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_message=True)

def add_message(input, output):
    memory.save_context({"input":input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

In [5]:
#add_message("Hi I'm a Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("South Korea is so pretty", "I wish I could go!!")

In [7]:
get_history()

{'history': "Human: Hi I'm a Nicolas, I live in South Korea\nAI: Wow that is so cool!\nHuman: South Korea is so pretty\nAI: I wish I could go!!"}

5. ConversationKGMemory
- 가장 중요한 정보만을 뽑은 요약본 느낌.

In [1]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_message=True)

def add_message(input, output):
    memory.save_context({"input":input}, {"output": output})

In [6]:
#add_message("Hi I'm a Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("Nicolas likes kimchi.", "Wow that is so cool!")

In [4]:
memory.load_memory_variables({"input": "who is Nicolas?"})

{'history': 'On Nicolas: Nicolas lives in South Korea.'}

In [7]:
memory.load_memory_variables({"input": "what does Nicolas like?"})

{'history': 'On Nicolas: Nicolas lives in South Korea. Nicolas likes kimchi.'}